In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PowerTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

In [2]:
df = pd.read_csv(r"spotify_songs.csv")

df.set_index('track_name', inplace=True)

df = df.drop_duplicates()

df = df.dropna()

In [3]:
numeric_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
X = df[numeric_columns]
y = pd.DataFrame(df['track_popularity'])

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y) 

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [6]:
#for kernel regression
kernels = [ConstantKernel(constant_value=1.0) + WhiteKernel(noise_level=1.0),
           ConstantKernel() * RBF(length_scale=1.0),
           ConstantKernel() * RBF() + WhiteKernel(),
           None #default added back
          ]

In [7]:
classifiers = [linear_model.LinearRegression(),
               linear_model.Lasso(), 
               linear_model.Ridge(), 
               KNeighborsRegressor()]
parameters = [{'fit_intercept':[True, False], 'positive':[True, False]},
            {'alpha':[1, 5, 10, 50, 100, 250, 500, 1000]}, 
              {'alpha':[0.5, 1, 5, 10, 50, 100, 250, 500]}, 
              {'n_neighbors':[1, 3, 5, 7, 9, 11], 'p':[1, 2, 3], 'weights':('uniform', 'distance')}]

In [8]:
folds = 5
jobs = -1

In [9]:
model_tracking = None

In [10]:
for i in np.arange(len(classifiers)):
    reg = GridSearchCV(classifiers[i], parameters[i], cv = folds, n_jobs = jobs)
    reg.fit(X_train, y_train)
    training_score = reg.best_score_
    test_score = reg.score(X_test, y_test)
    best_pars = str(reg.best_params_)
    test_predictions = reg.predict(X_test)
    test_mse = mean_squared_error(y_test, test_predictions)
    
    new_row = pd.DataFrame({
    'Cross Val Training Score': training_score,
    'Test Score': test_score,
    'MSE': test_mse,
    'Paramaters': best_pars},index=[classifiers[i]])
    model_tracking=pd.concat([model_tracking, new_row])

In [11]:
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

reg = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=folds,
                           n_jobs=jobs,
                           verbose=2)

reg.fit(X_train, y_train)
training_score = reg.best_score_
test_score = reg.score(X_test, y_test)
best_pars = str(reg.best_params_)
test_predictions = reg.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)

new_row = pd.DataFrame({
'Cross Val Training Score': training_score,
'Test Score': test_score,
'MSE': test_mse,
'Paramaters': best_pars},index=['Forest Regressor'])
model_tracking=pd.concat([model_tracking, new_row])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [12]:
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

reg = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=folds,
                           n_jobs=jobs,
                           verbose=2)

reg.fit(X_train, y_train)
training_score = reg.best_score_
test_score = reg.score(X_test, y_test)
best_pars = str(reg.best_params_)
test_predictions = reg.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)

new_row = pd.DataFrame({
'Cross Val Training Score': training_score,
'Test Score': test_score,
'MSE': test_mse,
'Paramaters': best_pars},index=['Gradient Boosting'])
model_tracking=pd.concat([model_tracking, new_row])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
C

In [13]:
pd.options.display.max_colwidth=99
model_tracking.round(2)

,Cross Val Training Score,Test Score,MSE,Paramaters
LinearRegression(),0.07,0.07,0.90,"{'fit_intercept': False, 'positive': False}"
Lasso(),-0.00,-0.00,0.97,{'alpha': 1}
Ridge(),0.07,0.07,0.90,{'alpha': 50}
KNeighborsRegressor(),0.18,0.18,0.79,"{'n_neighbors': 11, 'p': 2, 'weights': 'distance'}"
Forest Regressor,0.28,0.27,0.70,"{'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}"
Gradient Boosting,0.11,0.11,0.86,"{'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2}"


In [14]:
from sklearn.ensemble import StackingRegressor
estimators = [
    ('KN Reg', KNeighborsRegressor(n_neighbors = 11, p = 2, weights = 'distance')),
    ('Forest Regressor', RandomForestRegressor(max_features=  'sqrt', min_samples_leaf = 1, min_samples_split = 2, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(max_features = 'auto', min_samples_leaf = 1, min_samples_split = 2, random_state=42))
]

reg = StackingRegressor(
    estimators=estimators,
    cv=folds,
    n_jobs=jobs)

reg.fit(X_train, y_train)
training_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)
test_predictions = reg.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)

new_row = pd.DataFrame({
'Cross Val Training Score': training_score,
'Test Score': test_score,
'MSE': test_mse,
'Paramaters': 'None'},index=['Stacked'])
model_tracking=pd.concat([model_tracking, new_row])

C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\ensemble\_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
pd.options.display.max_colwidth=99
model_tracking.round(2)

,Cross Val Training Score,Test Score,MSE,Paramaters
LinearRegression(),0.07,0.07,0.90,"{'fit_intercept': False, 'positive': False}"
Lasso(),-0.00,-0.00,0.97,{'alpha': 1}
Ridge(),0.07,0.07,0.90,{'alpha': 50}
KNeighborsRegressor(),0.18,0.18,0.79,"{'n_neighbors': 11, 'p': 2, 'weights': 'distance'}"
Forest Regressor,0.28,0.27,0.70,"{'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}"
Gradient Boosting,0.11,0.11,0.86,"{'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2}"
Stacked,0.90,0.28,0.70,None


In [20]:
from sklearn.ensemble import StackingRegressor
estimators = [
    ('KN Reg', KNeighborsRegressor(n_neighbors = 11, p = 2, weights = 'distance')),
    ('Forest Regressor', RandomForestRegressor(max_features=  'sqrt', min_samples_leaf = 1, min_samples_split = 2, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(max_features = 'auto', min_samples_leaf = 1, min_samples_split = 2, random_state=42))
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator = linear_model.Ridge(alpha=50),
    cv=folds,
    n_jobs=jobs)

reg.fit(X_train, y_train)
training_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)
test_predictions = reg.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)

new_row = pd.DataFrame({
'Cross Val Training Score': training_score,
'Test Score': test_score,
'MSE': test_mse,
'Paramaters': 'None'},index=['Stacked Ridge'])
model_tracking=pd.concat([model_tracking, new_row])

C:\Users\kulul\anaconda3\Lib\site-packages\sklearn\ensemble\_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
pd.options.display.max_colwidth=99
model_tracking.round(2)

,Cross Val Training Score,Test Score,MSE,Paramaters
LinearRegression(),0.07,0.07,0.90,"{'fit_intercept': False, 'positive': False}"
Lasso(),-0.00,-0.00,0.97,{'alpha': 1}
Ridge(),0.07,0.07,0.90,{'alpha': 50}
KNeighborsRegressor(),0.18,0.18,0.79,"{'n_neighbors': 11, 'p': 2, 'weights': 'distance'}"
Forest Regressor,0.28,0.27,0.70,"{'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}"
Gradient Boosting,0.11,0.11,0.86,"{'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2}"
Stacked,0.90,0.28,0.70,None
Stacked Lasso,0.00,-0.00,0.97,None
Stacked Ridge,0.90,0.28,0.70,None
